# DAT257x: Reinforcement Learning Explained

## Lab 4: Dynamic Programming

### Exercise 4.1 Policy Evaluation with 2 Arrays

Policy Evaluation calculates the value function for a policy, given the policy and the full definition of the associated Markov Decision Process.  The full definition of an MDP is the set of states, the set of available actions for each state, the set of rewards, the discount factor, and the state/reward transition function.

In [1]:
import test_dp               # required for testing and grading your code
import gridworld_mdp as gw   # defines the MDP for a 4x4 gridworld

The gridworld MDP defines the probability of state transitions for our 4x4 gridworld using a "get_transitions()" function.  

Let's try it out now, with state=2 and all defined actions.

In [7]:
# try out the gw.get_transitions(state, action) function

state = 2
actions = gw.get_available_actions(state)

for action in actions:
    transitions = gw.get_transitions(state=state, action=action)

    # examine each return transition (only 1 per call for this MDP)
    #for (trans) in transitions:
    next_state, reward, probability = transitions[0]    # unpack tuple
    print("transition("+ str(state) + ", " + action + "):", "next_state=", next_state, ", reward=", reward, ", probability=", probability)

transition(2, up): next_state= 2 , reward= -1 , probability= 1
transition(2, down): next_state= 6 , reward= -1 , probability= 1
transition(2, left): next_state= 1 , reward= -1 , probability= 1
transition(2, right): next_state= 3 , reward= -1 , probability= 1


**Implement the algorithm for Iterative Policy Evaluation using the 2 array approach**.  In the 2 array approach, one array holds the value estimates for each state computed on the previous iteration, and one array holds the value estimates for the states computing in the current iteration.

A empty function **policy_eval_two_arrays** is provided below; implement the body of the function to correctly calculate the value of the policy using the 2 array approach.  The function defines 5 parameters - a definition of each parameter is given in the comment block for the function.  For sample parameter values, see the calling code in the cell following the function.

In [68]:
def policy_eval_two_arrays(state_count, gamma, theta, get_policy, get_transitions):
    """
    This function uses the two-array approach to evaluate the specified policy for the specified MDP:
    
    'state_count' is the total number of states in the MDP. States are represented as 0-relative numbers.
    
    'gamma' is the MDP discount factor for rewards.
    
    'theta' is the small number threshold to signal convergence of the value function (see Iterative Policy Evaluation algorithm).
    
    'get_policy' is the stochastic policy function - it takes a state parameter and returns list of tuples, 
        where each tuple is of the form: (action, probability).  It represents the policy being evaluated.
        
    'get_transitions' is the state/reward transiton function.  It accepts two parameters, state and action, and returns
        a list of tuples, where each tuple is of the form: (next_state, reward, probabiliity).  
        
    """
    V = state_count*[0]
    while True:
        delta = 0
        _v = V[:]
        for i in range(1,len(V)):
            a = V[i]
            tt = 0
            for act,prob in get_policy(i):
                trans = get_transitions(state=i,action=act)[0]
                tt += prob*(trans[1] + gamma*_v[trans[0]]) #accumlation variant, it must be 0 when initization
                V[i] = tt
            delta = max(delta,abs(a - V[i]))
        if delta < theta:
            print("finish")
            break
    return V

First, test our function using the MDP defined by gw.* functions.

In [66]:
def get_equal_policy(state):
    # build a simple policy where all 4 actions have the same probability, ignoring the specified state
    policy = ( ("up", .25), ("right", .25), ("down", .25), ("left", .25))
    return policy

n_states = gw.get_state_count()

# test our function
values = policy_eval_two_arrays(state_count=n_states, gamma=.9, theta=.001, get_policy=get_equal_policy, \
    get_transitions=gw.get_transitions)

print("Values=", values)

[0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
[0, -1.6749999999999998, -1.9, -1.9, -1.6749999999999998, -1.9, -1.9, -1.9, -1.9, -1.9, -1.9, -1.6749999999999998, -1.9, -1.9, -1.6749999999999998]
[0, -2.2318749999999996, -2.659375, -2.71, -2.231875, -2.6087499999999997, -2.71, -2.659375, -2.659375, -2.71, -2.6087499999999997, -2.2318749999999996, -2.71, -2.659375, -2.2318749999999996]
[0, -2.6875, -3.3200312500000004, -3.4162187500000005, -2.6875, -3.2238437500000003, -3.37065625, -3.32003125, -3.3200312500000004, -3.3706562499999997, -3.22384375, -2.6875, -3.41621875, -3.32003125, -2.6875]
[0, -3.077059375, -3.8787414062500005, -4.0313125, -3.077059375, -3.7261703125, -3.94474375, -3.87874140625, -3.8787414062500005, -3.9447437499999998, -3.7261703125000003, -3.077059375, -4.0313125, -3.8787414062500005, -3.077059375]
[0, -3.4034434960937503, -4.359667832031251, -4.5595242578125, -3.4034434960937503, -4.15981140625, -4.422210273437501, -4.359667

**Expected output from running above cell:**

`
Values= [0.0, -5.274709263277986, -7.123800104889248, -7.64536148969558, -5.274709263277987, -6.602238720082915, -7.17604178238719, -7.1238001048892485, -7.1238001048892485, -7.176041782387191, -6.602238720082915, -5.274709263277986, -7.645361489695581, -7.1238001048892485, -5.274709263277986]
`

In [69]:
import numpy as np
a = np.append(values, 0)
np.reshape(a, (4,4))

array([[ 0.        , -5.27470926, -7.1238001 , -7.64536149],
       [-5.27470926, -6.60223872, -7.17604178, -7.1238001 ],
       [-7.1238001 , -7.17604178, -6.60223872, -5.27470926],
       [-7.64536149, -7.1238001 , -5.27470926,  0.        ]])

Now, test our function using the test_dp helper.  The helper also uses the gw MDP, but with a different gamma value.
If our function passes all tests, a passcode will be printed.

In [70]:
# test our function using the test_db helper
test_dp.policy_eval_two_arrays_test( policy_eval_two_arrays ) 


Testing: Policy Evaluation (two-arrays)
finish
passed test: return value is list
passed test: length of list = 15
passed test: values of list elements
PASSED: Policy Evaluation (two-arrays) passcode = 9986-145
